In [ ]:
import os
from pathlib import Path
from io import StringIO
import dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureChatOpenAI

dotenv.load_dotenv()

In [ ]:
model: AzureChatOpenAI = None
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
Look at the image below and use it as input. Generate a response based on the image.
You are an expert in ontology engineering. Generate an OWL ontology based on the following domain description:
Define classes, data properties, and object properties with their designation, the norm name, a yes/no if the standard has been withdrawn and the respective name of the ISO/DIN norm.
Include domain and range for each property.
Provide the output in OWL (XML) format and only output the ontology and nothing else"""

task_prompt = """Generate an ontology based on the following domain description of screw specification in ISO and DIN norm. This is the existing ontology that you need to extend if necessary:

"""

In [ ]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

model = AzureChatOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
        openai_api_version=os.getenv("AZURE_OPENAI_VERSION"),
        temperature=0
    )

In [ ]:
import base64
from io import BytesIO
from PIL import Image

def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Base64 string
    """
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def load_image(image_path):
    """
    Load an image from a file path

    :param image_path: Path to the image
    :return: PIL image
    """
    return Image.open(image_path)

In [ ]:
image_path = "../../data/Broschuere_DIN-EN-ISO_Normung_DE"

onthology_file_path="onthology_bolts.xml"

def process_file(image_file_path, onthology_file_path):
    image_data = convert_to_base64(load_image(image_file_path))
    with open(onthology_file_path, "r") as file:
        ontology_content = file.read()

    message = HumanMessage(
        content=[
            {"type": "text", "text": system_prompt + task_prompt + ontology_content},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{image_data}"},
            },
        ],
    )

    response = model.invoke([message])
    print(response.content)

    with open(onthology_file_path, "w") as file:
        new_ontology = response.content.replace("```xml", "").replace("```", "")
        file.write(new_ontology)

folder = Path(image_path)

files = [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]
files.sort()
for file in files:
    file_path = Path(folder, file)
    print(file_path)
    process_file(file_path, onthology_file_path)
